<a href="https://colab.research.google.com/github/mattiadido95/Haze-Fog-suppression/blob/main/pix2pixHD_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [1]:
!pip install dominate gdown

In [2]:
from google.colab import drive
import os
import gdown

In [3]:
# mount drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
folder_path = "drive/MyDrive/Haze-Fog-suppression"
os.chdir(folder_path)

# Install Apex for Automatic Mixed Precision to speed up training


In [ ]:
#!git clone https://github.com/NVIDIA/apex
%cd apex
# if pip >= 23.1 (ref: https://pip.pypa.io/en/stable/news/#v23-1) which supports multiple `--config-settings` with the same key...
!pip install -v --disable-pip-version-check --no-cache-dir --no-build-isolation --config-settings "--build-option=--cpp_ext" --config-settings "--build-option=--cuda_ext" ./
%cd ..

/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/apex
Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Processing /content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/apex
  Running command Preparing metadata (pyproject.toml)


  torch.__version__  = 2.0.1+cu118


  running dist_info
  creating /tmp/pip-modern-metadata-k24uqvr5/apex.egg-info
  writing /tmp/pip-modern-metadata-k24uqvr5/apex.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-modern-metadata-k24uqvr5/apex.egg-info/dependency_links.txt
  writing requirements to /tmp/pip-modern-metadata-k24uqvr5/apex.egg-info/requires.txt
  writing top-level names to /tmp/pip-modern-metadata-k24uqvr5/apex.egg-info/top_level.txt
  writing manifest file '/tmp/pip-modern-metadata-k24uqvr5/apex.egg-info/SOURCES.txt'
  reading manifest file '/tmp/pip-modern-metadata-k24uqvr5/apex.egg-info/SOURCES.txt'
  adding license file 'LICENSE'
  writing manifest file '/tmp/pip-modern-metadata-k24uqvr5/apex.egg-inf

# Pix2PixHD download


In [ ]:
!git clone https://github.com/NVIDIA/pix2pixHD.git
os.chdir("pix2pixHD")

Cloning into 'pix2pixHD'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 55.68 MiB | 20.18 MiB/s, done.
Resolving deltas: 100% (156/156), done.
Updating files: 100% (115/115), done.
/content/drive/MyDrive/Haze-suppression/pix2pixHD


In [ ]:
os.makedirs("./checkpoints/label2city_1024p/")
os.chdir("./checkpoints/label2city_1024p/")

In [ ]:
url = 'https://drive.google.com/u/0/uc?id=1h9SykUnuZul7J3Nbms2QGH1wa85nbN2-&export=download'
output = 'latest_net_G.pth'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/u/0/uc?id=1h9SykUnuZul7J3Nbms2QGH1wa85nbN2-&export=download
To: /content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/checkpoints/label2city_1024p/latest_net_G.pth
100%|██████████| 732M/732M [00:11<00:00, 62.1MB/s]


'latest_net_G.pth'

In [ ]:
os.chdir("../..")

# Pix2PixHD Train


In [5]:
%cd pix2pixHD/

/content/drive/.shortcut-targets-by-id/1MeMU7hWD5WVMk25vU3ch5Ii3csnSydwf/Haze-Fog-suppression/pix2pixHD


In [6]:
#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:1000"

#training on low resolution only G1
!python train.py --continue_train --label_nc 0 --no_instance --name nebbia --dataroot ./datasets/nebbia --resize_or_crop crop --fineSize 512 --batchSize 4

------------ Options -------------
batchSize: 4
beta1: 0.5
checkpoints_dir: ./checkpoints
continue_train: True
data_type: 32
dataroot: ./datasets/nebbia
debug: False
display_freq: 100
display_winsize: 512
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
input_nc: 3
instance_feat: False
isTrain: True
label_feat: False
label_nc: 0
lambda_feat: 10.0
loadSize: 1024
load_features: False
load_pretrain: 
local_rank: 0
lr: 0.0002
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_layers_D: 3
n_local_enhancers: 1
name: nebbia
ndf: 64
nef: 16
netG: global
ngf: 64
niter: 100
niter_decay: 100
niter_fix_global: 0
no_flip: False
no_ganFeat_loss: False
no_html: False
no_instance: True
no_lsgan: False
no_vgg_loss: False
norm: instance
num_D: 2
output_nc: 3
phase: train
pool_size: 0
print_freq: 100
resize_or_crop: crop
save_epoch_freq: 10
save_latest_freq: 1000
serial_batches: False
tf_log: False
use_dropou

In [ ]:
#training on high resolution G1 and G2
#!python train.py --netG local --load_pretrain checkpoints/nebbia/ --label_nc 0 --no_instance --name nebbia --dataroot ./datasets/nebbia --resize_or_crop crop --fineSize 1024

# Pix2Pix test

In [ ]:
#se non è stato eseguito il download
if os.getcwd() != "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD":
  os.chdir("pix2pixHD")

#!./scripts/test_1024p.sh
!python test.py --name nebbia --netG local --ngf 32 --resize_or_crop none --ntest 5

iter.txt  latest_net_D.pth  latest_net_G.pth  loss_log.txt  opt.txt  web
Traceback (most recent call last):
  File "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/test.py", line 12, in <module>
    opt = TestOptions().parse(save=False)
  File "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/options/base_options.py", line 80, in parse
    torch.cuda.set_device(self.opt.gpu_ids[0])
  File "/usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py", line 350, in set_device
    torch._C._cuda_setDevice(device)
  File "/usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py", line 247, in _lazy_init
    torch._C._cuda_init()
RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx


In [ ]:
#!cp -r /content/drive/MyDrive/Haze-Fog-suppression/dataset/SOTS/outdoor/gt/* /content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/datasets/nebbia/test_img/

In [ ]:
#!cp -r /content/drive/MyDrive/Haze-Fog-suppression/dataset/SOTS/outdoor/hazy/* /content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/datasets/nebbia/test_label/